Analyze state of the union addresses. 
Data source: https://en.wikisource.org/wiki/Portal:State_of_the_Union_Speeches_by_United_States_Presidents
scrape the text of all speeches and then maybe try to find patterns of speech of each president?

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib.request

# Data source we are going to scrape for results
data_url = 'https://en.wikisource.org/wiki/Portal:State_of_the_Union_Speeches_by_United_States_Presidents'

link_list = []

resp = urllib.request.urlopen(data_url)
soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))

# Get all links to state of the union addresses from 
for link in soup.find_all('a', href=True):
    if "union_address" in link['href'].lower() or "union_speech" in link['href'].lower() \
        and "portal" not in link['href'].lower() and "#" not in link['href'].lower():
        link_list.append(link['href'])

# extract the text of a speech from a URL
# text is extracted in a list of paragraphs (strings) for each speech
def get_speech(url):
    return([ p.text.strip() for p in BeautifulSoup(urllib.request.urlopen(url)).find_all("p") if \
             'This work is in the public domain in the United States because it is a work of the United States federal government' \
             not in p.text.strip()])

speeches = [get_speech('https://en.wikisource.org' + link) for link in link_list]

In [63]:
# Extract presidents names from link text
presidents = [ link.replace('%','/').split('/')[2].replace('_',' ') for link in link_list ]

# Extract state of the union text entries so we can extract the date
sou_entries = []

for item in soup.find_all('li'):
    if 'union' in item.text.strip().lower() and '(' in  item.text.strip().lower():
        sou_entries.append(item.text.strip())


In [64]:
print(sou_entries[0])
import re
re.findall('\d+',sou_entries[0])

First State of the Union Address - George Washington, (8 January 1790)


['8', '1790']

In [70]:
speeches_pd = pd.DataFrame({
                'president' : presidents,
                'speech' : speeches,
                'year' : [int(re.findall('\d+',item)[1]) for item in sou_entries ]} )

In [73]:
speeches_pd.head(10)

,president,speech,year
0,George Washington,[I embrace with great satisfaction the opportu...,1790
1,George Washington,[Fellow-Citizens of the Senate and the House o...,1790
2,George Washington,[Fellow-Citizens of the Senate and the House o...,1791
3,George Washington,[Fellow-Citizens of the Senate and of the Hous...,1792
4,George Washington,[Fellow Citizens of the Senate and of the Hous...,1793
5,George Washington,[Fellow Citizens of the Senate and of the Hous...,1794
6,George Washington,[Fellow Citizens of the Senate and of the Hous...,1795
7,George Washington,[Fellow Citizens of the Senate and of the Hous...,1796
8,John Adams,[I was for some time apprehensive that it woul...,1797
9,John Adams,[Gentlemen of the Senate and Gentlemen of the ...,1798


In [ ]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import pandas as pd